In [ ]:
from traitlets.traitlets import Set
#Sort the k-mers alphabetically
def sort(read1,read2,num_frequent_k_mers):
    size=len(read1)+len(read2)
     #key is read_ID+k-mer index
    with_ID_read1={}
    for key in read1:
      k=str(1)+" "+str(key)
      with_ID_read1[k]=read1[key]
    with_ID_read2={}
    for key in read2:
      k=str(2)+" "+str(key)
      with_ID_read2[k]=read2[key]
    with_ID_read1.update(with_ID_read2)
    #Sort the merged dictionary;key is the index in the unsorted dictionary
    sorted_keys = sorted(with_ID_read1, key=with_ID_read1.get)
    #sorted_merged_reads=sorted(with_ID_read1)
    sorted_merged_reads={}
    for w in sorted_keys:
       sorted_merged_reads[w] = with_ID_read1[w]
    #Eliminate the k_mers that occur too often
    sorted_merged_reads=eliminate(sorted_merged_reads,num_frequent_k_mers)
    table={}
    unique_k_mer=sorted(set(list(sorted_merged_reads.values())))
    #for each k_mer, find its occurences in both reads
    for k_mer in unique_k_mer:
      read_1_indices=[]
      read_2_indices=[]
      #Get a set of positions of the k_mers
      value = {i for i in sorted_merged_reads if sorted_merged_reads[i]==k_mer}
      
      #for each position in the set
      for key in value:
          #split the key into read_ID and index
          split_key=key.split(" ")
          #print(split_key)
          if split_key[0]=='1':
            read_1_indices.append(split_key[1])
          else:
            read_2_indices.append(split_key[1])
      size=max(len(read_1_indices),len(read_2_indices))
      values=np.zeros([2,size])
      values[0,0:len(read_1_indices)]=read_1_indices
      values[1,0:len(read_2_indices)]=read_2_indices
      table[k_mer]=values
    return table



In [ ]:
#Eliminate the k_mers that occur too often
from collections import Counter
def eliminate(merged_dict,number):
  k_mers=list(merged_dict.values())  
  #count the occurence of all k_mers
  counts = Counter(k_mers)
  #return the n most common k_mers
  most_common=counts.most_common(number)
  for i in range(number):
      merged_dict = {key:val for key, val in merged_dict.items() if val != most_common[i][0]}
  return merged_dict